# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [24]:
nutrition_agent = Agent(
    name="Nutrition agent",
    instructions="""
    you are helpul
    you give concise answers
    """
)

In [29]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition agent", ...)
- Final output (str):
    Overall, yes—bananas are a healthy, convenient snack.
    
    What you get per medium banana (~105 calories):
    - Potassium ~422 mg (good for heart and blood pressure)
    - Vitamin B6, vitamin C
    - 3 g fiber (plus some resistant starch when green)
    - ~27 g carbohydrates (natural sugars)
    
    Benefits:
    - Supports heart and digestion
    - Low fat, portable, affordable
    
    Notes:
    - Ripeness affects sugar and GI (ripe = sweeter; greener = higher resistant starch, potentially lower GI)
    - Not a complete protein; pair with other foods for balanced meals
    - If you have kidney disease or very high potassium concerns, check with a clinician
    
    Bottom line: a nutritious, versatile part of a balanced diet.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [30]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are healthy for most people. Key points:

- Nutrients: good source of potassium, vitamin B6, vitamin C, fiber.
- Health benefits: supports heart health, digestion, and energy; resistant starch in underripe bananas may aid gut health.
- Considerations: moderate sugar and calories (about 100 kcal per medium); ripe bananas have higher sugar and GI than green ones.
- Special cases: people with kidney disease may need to limit potassium; allergies or oral allergy syndrome can occur in rare cases.
- Tips: pair with protein/fat (e.g., peanut butter) to curb sugar spike; store ripe if you want sweeter taste, green if you want slower ripening.

_Good Job!_